In [1]:
# Processing tools to read from Helsinki WFS API and produce GeoJSON

import json

import geojson
import requests


# Helsinki address data


In [2]:
address_object_sample = {
    "type": "Feature",
    "id": "Helsinki_osoiteluettelo.46991",
    "geometry": {
        "type": "Point",
        "coordinates": [
            24.95061169,
            60.16659673
        ]
    },
    "geometry_name": "geom",
    "properties": {
        "id": 46991,
        "katunimi": "Unioninkatu",
        "gatan": "Unionsgatan",
        "osoitenumero": 24,
        "osoitenumero2": None,
        "osoitekirjain": None,
        "n": 6672635,
        "e": 25497258,
        "kaupunki": "Helsinki",
        "staden": "Helsingfors",
        "tyyppi": 1,
        "postinumero": "00130",
        "postitoimipaikka": "Helsinki",
        "osoitenumero_teksti": "24",
        "datanomistaja": None,
        "paivitetty_tietopalveluun": "2023-04-25"
    }
}


def cleanup_address_object(obj):
    """
    Remove unnecessary fields from the address object and rename some.
    """
    obj.pop("geometry_name")
    obj["properties"]["name"] = {
        "fi": obj["properties"]["katunimi"],
        "sv": obj["properties"]["gatan"],
    }
    obj["properties"].pop("katunimi")
    obj["properties"].pop("gatan")
    properties_map = {
        "osoitenumero": "street_number",
        "osoitenumero2": "street_number2",
        "osoitekirjain": "street_letter",
        "osoitenumero_teksti": "street_number_text",
        "kaupunki": "city",
        "postinumero": "postal_code",
        "postitoimipaikka": "postal_code_name",
    }
    for k, v in properties_map.items():
        obj["properties"][v] = obj["properties"][k]
        obj["properties"].pop(k)
    properties_to_remove = [
        "tyyppi",
        "staden",
        "datanomistaja",
        "paivitetty_tietopalveluun",
    ]
    for k in properties_to_remove:
        obj["properties"].pop(k)
    return obj


In [3]:
# Helsinki Service Map data and save to GeoJSON files
address_url = "https://kartta.hel.fi/ws/geoserver/avoindata/wfs?service=wfs&version=2.0.0&request=GetFeature&typeNames=avoindata:Helsinki_osoiteluettelo&count=1000000&outputFormat=application/json&srsName=urn:ogc:def:crs:EPSG::4326"

res = requests.get(address_url)
address_geojson = res.json()
features = []
# Loop all featues and clean up the data
for feature in address_geojson["features"]:
    features.append(cleanup_address_object(feature))

geojson_obj = geojson.FeatureCollection(features)
fname = "HelsinkiAddressList.json"
with open(fname, "w") as f:
    json.dump(geojson_obj, f)
print(f"* [{fname}](https://proto.fvh.io/urbanage/{fname})")



* [HelsinkiAddressList.geojson](https://proto.fvh.io/urbanage/HelsinkiAddressList.geojson)
